In [47]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score

import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [48]:
# Loads the AMES housing Dataset
housing = fetch_openml(name="house_prices", as_frame=True)  # noqa

In [49]:
data = pd.DataFrame(data= np.c_[housing['data'], housing['target']],
                     columns= housing['feature_names'] + ['target'])

In [50]:
liste_to_numeric = ['YrSold', 'YearBuilt', 'GrLivArea', 'LotFrontage', 'LotArea','GarageArea']
data[liste_to_numeric] = data[liste_to_numeric].apply(pd.to_numeric)

In [51]:
data = data*1

Data cleaning

In [52]:
# create dumies
data['Fence'] = data['Fence'].notna()
data['Pool'] = data['PoolArea'] > 0

# create the new variable 'age'
data['Age'] = data.apply(lambda x: x['YrSold']-x['YearBuilt'] if (x['YearBuilt']<x['YearRemodAdd']) 
                                                           else (x['YrSold']-x['YearRemodAdd']), axis=1)

# Delete all data with MSZoning = commercial, agriculture and industrial as these are not residential units
data = data[(data.MSZoning != 'C (all)') & (data.MSZoning != 'I (all)') & (data.MSZoning != 'A (agr)')]

# Delete abnormal sales
data = data[(data.SaleCondition != 'Abnorml')] 
data = data[(data.SaleCondition != 'Family')] 


In [53]:
# Select Features
X = data[['Age','GrLivArea', 'LotFrontage', 'LotArea', 'GarageArea', 'Fence', 'Pool']]
Y = data[['target']]

In [54]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=.3, random_state=1121218)

Evaluation du modele

In [55]:
first_model = pickle.load(open("finalized_model.sav", 'rb'))

In [56]:
predict_on_Xtrain = first_model.predict(X_train)
predict_on_Xvalid = first_model.predict(X_valid)

In [57]:
r2_train = r2_score(y_train,predict_on_Xtrain)
r2_valid = r2_score(y_valid,predict_on_Xvalid)
mae_train = mean_absolute_error(y_train,predict_on_Xtrain)
mae_valid = mean_absolute_error(y_valid,predict_on_Xvalid)

In [58]:
print("SUR LES DONNEES D'ENTRAINEMENT:")
print("Le modèle prédit une valeur exacte dans {} % des cas".format(round(r2_train*100,2)))
print("et ce avec une erreure moyenne de {}$, ce qui est un delta acceptable sur le prix d'un bien immobilier.".format(round(mae_train)))

print("SUR LES DONNEES DE VALIDATION:")
print("Le modèle prédit une valeur exacte dans {} % des cas".format(round(r2_valid*100,2)))
print("et ce avec une erreure moyenne de {}$, ce qui n'est plus acceptable.".format(round(mae_valid)))

print("CONCLUSION:")
print("l'écart de performance des predictions est trop imortant, on constate un overfitting du model")

SUR LES DONNEES D'ENTRAINEMENT:
Le modèle prédit une valeur exacte dans 96.81 % des cas
et ce avec une erreure moyenne de 8863$, ce qui est un delta acceptable sur le prix d'un bien immobilier.
SUR LES DONNEES DE VALIDATION:
Le modèle prédit une valeur exacte dans 77.21 % des cas
et ce avec une erreure moyenne de 22350$, ce qui n'est plus acceptable.
CONCLUSION:
l'écart de performance des predictions est trop imortant, on constate un overfitting du model
